## Data Model Implementation

In [44]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report
from scipy.stats import randint as sp_randint
import joblib
import tensorflow as tf

# Load the dataset
df = pd.read_csv('heart_disease.csv')

# Check for missing values and fill them
df.fillna(df.mean(), inplace=True)


df.head(5)

,HeartDiseaseorAttack,HighBP,HighChol,CholCheck,BMI,Smoker,Stroke,Diabetes,PhysActivity,Fruits,...,AnyHealthcare,NoDocbcCost,GenHlth,MentHlth,PhysHlth,DiffWalk,Sex,Age,Education,Income
0,0.0,1.0,1.0,1.0,40.0,1.0,0.0,0.0,0.0,0.0,...,1.0,0.0,5.0,18.0,15.0,1.0,0.0,9.0,4.0,3.0
1,0.0,0.0,0.0,0.0,25.0,1.0,0.0,0.0,1.0,0.0,...,0.0,1.0,3.0,0.0,0.0,0.0,0.0,7.0,6.0,1.0
2,0.0,1.0,1.0,1.0,28.0,0.0,0.0,0.0,0.0,1.0,...,1.0,1.0,5.0,30.0,30.0,1.0,0.0,9.0,4.0,8.0
3,0.0,1.0,0.0,1.0,27.0,0.0,0.0,0.0,1.0,1.0,...,1.0,0.0,2.0,0.0,0.0,0.0,0.0,11.0,3.0,6.0
4,0.0,1.0,1.0,1.0,24.0,0.0,0.0,0.0,1.0,1.0,...,1.0,0.0,2.0,3.0,0.0,0.0,0.0,11.0,5.0,4.0


In [45]:
# Exclude AnyHealthcare NoDocbcCost and Income columns
df = df.drop(['AnyHealthcare', 'NoDocbcCost','Income'], axis=1)

# Splitting the dataset into features (X) and target (y)
X = df.drop('HeartDiseaseorAttack', axis=1)
y = df['HeartDiseaseorAttack']

# Normalize and standardize the data
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

In [53]:
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

# Initialize and train the model with RandomizedSearchCV
param_dist = {
    'n_estimators': sp_randint(100, 200),
    'max_depth': sp_randint(5, 20),
    'min_samples_split': sp_randint(2, 11),
    'min_samples_leaf': sp_randint(1, 11),
    'bootstrap': [True, False]
}

random_search = RandomizedSearchCV(RandomForestClassifier(random_state=42), param_distributions=param_dist, n_iter=50, cv=3, random_state=42, n_jobs=-1)
random_search.fit(X_train, y_train)

best_model = random_search.best_estimator_

# Save the model and scaler
joblib.dump(best_model, 'heart_disease_model.pkl')
joblib.dump(scaler, 'scaler.pkl')

KeyboardInterrupt: 

In [5]:
# Evaluate the model on the test set
y_pred = best_model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
report = classification_report(y_test, y_pred)

print(f"Accuracy: {accuracy}")
print(f"Classification Report:\n{report}")

Accuracy: 0.9080140334279407
Classification Report:
              precision    recall  f1-score   support

         0.0       0.91      1.00      0.95     45968
         1.0       0.60      0.06      0.12      4768

    accuracy                           0.91     50736
   macro avg       0.75      0.53      0.53     50736
weighted avg       0.88      0.91      0.87     50736



In [6]:
# Save the feature importance plot
import matplotlib.pyplot as plt
import os

def save_feature_importances(model, feature_names, output_file):
    if not os.path.exists(os.path.dirname(output_file)):
        os.makedirs(os.path.dirname(output_file))
    importances = model.feature_importances_
    indices = np.argsort(importances)

    plt.figure(figsize=(10, 8))
    plt.title('Feature Importances')
    plt.barh(range(len(indices)), importances[indices], align='center')
    plt.yticks(range(len(indices)), [feature_names[i] for i in indices])
    plt.xlabel('Relative Importance')
    plt.savefig(output_file)
    plt.close()

feature_names = X.columns.tolist()
save_feature_importances(best_model, feature_names, 'static/feature_importances.png')

## Data Model Optimisation

In [47]:
# Splitting the dataset into features (X) and target (y)
X = df.drop('HeartDiseaseorAttack', axis=1).values
y = df['HeartDiseaseorAttack'].values

In [48]:
# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

# Normalize and standardize the data
scaler = StandardScaler()

# Scale the data
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [49]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
number_input_features = len(X_train[0])
hidden_nodes_layer1 =  20
hidden_nodes_layer2 = 12

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu")
)
# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()

/Users/racheljiang/anaconda3/envs/dev/lib/python3.10/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_6 (Dense)                 │ (None, 20)             │           380 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_7 (Dense)                 │ (None, 12)             │           252 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_8 (Dense)                 │ (None, 1)              │            13 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 645 (2.52 KB)

 Trainable params: 645 (2.52 KB)

 Non-trainable params: 0 (0.00 B)

In [50]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [51]:
# Train the model
fit_model = nn.fit(X_train_scaled,y_train,epochs=50)

Epoch 1/50
5946/5946 ━━━━━━━━━━━━━━━━━━━━ 2s 290us/step - accuracy: 0.9042 - loss: 0.2591
Epoch 2/50
5946/5946 ━━━━━━━━━━━━━━━━━━━━ 2s 295us/step - accuracy: 0.9068 - loss: 0.2388
Epoch 3/50
5946/5946 ━━━━━━━━━━━━━━━━━━━━ 2s 301us/step - accuracy: 0.9071 - loss: 0.2388
Epoch 4/50
5946/5946 ━━━━━━━━━━━━━━━━━━━━ 2s 309us/step - accuracy: 0.9085 - loss: 0.2362
Epoch 5/50
5946/5946 ━━━━━━━━━━━━━━━━━━━━ 2s 312us/step - accuracy: 0.9070 - loss: 0.2383
Epoch 6/50
5946/5946 ━━━━━━━━━━━━━━━━━━━━ 2s 301us/step - accuracy: 0.9073 - loss: 0.2389
Epoch 7/50
5946/5946 ━━━━━━━━━━━━━━━━━━━━ 2s 296us/step - accuracy: 0.9068 - loss: 0.2397
Epoch 8/50
5946/5946 ━━━━━━━━━━━━━━━━━━━━ 2s 288us/step - accuracy: 0.9073 - loss: 0.2379
Epoch 9/50
5946/5946 ━━━━━━━━━━━━━━━━━━━━ 2s 292us/step - accuracy: 0.9072 - loss: 0.2375
Epoch 10/50
5946/5946 ━━━━━━━━━━━━━━━━━━━━ 2s 292us/step - accuracy: 0.9088 - loss: 0.2354
Epoch 11/50
5946/5946 ━━━━━━━━━━━━━━━━━━━━ 2s 284us/step - accuracy: 0.9067 - loss: 0.2373
Epoch 12

In [52]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

1982/1982 - 0s - 235us/step - accuracy: 0.9090 - loss: 0.2365
Loss: 0.23645345866680145, Accuracy: 0.9090192317962646
